# Imports

In [ ]:
import tensorflow as tf

# Datasets in Tensorflow

### TFRecordDataset

In [ ]:
my_tfrecords_dataset = tf.data.TFRecordDataset(["file1.tfrecords", "file2.tfrecords"])

### from_tensor_slices

In [ ]:
my_dataset = tf.data.Dataset.from_tensor_slices([2,3,6])
my_dataset = my_dataset.map(lambda x:x+2)

for datapoint in my_dataset:
  print(datapoint)

image1 = [0,0,0,0]
image2 = [1,1,1,1]
images = [image1, image2] 

y1 = 0
y2 = 1
labels = [y1, y2]

my_dataset2 = tf.data.Dataset.from_tensor_slices((images, labels))

for datapoint in my_dataset2:
  print(datapoint)

tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
(<tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 0, 0, 0], dtype=int32)>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(4,), dtype=int32, numpy=array([1, 1, 1, 1], dtype=int32)>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)


### from_generator

In [ ]:
my_dataset3 = tf.data.Dataset.from_generator(generator)
#data augmentation

### tf.keras.datasets

In [ ]:
my_dataset4 = tf.keras.datasets.mnist.load_data()

(x_train, y_train), (x_test, y_test) = my_dataset4

print("x_train.shape = ", x_train.shape)
print("y_train.shape = ", y_train.shape)
print("x_test.shape = ", x_test.shape)
print("y_test.shape = ", y_test.shape)

11501568/11490434 [==============================] - 0s 0us/step
x_train.shape =  (60000, 28, 28)
y_train.shape =  (60000,)
x_test.shape =  (10000, 28, 28)
y_test.shape =  (10000,)


# Deep Learning Layers in Tensorflow

## Accessing keras layers

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D

## Input layer

In [ ]:
# Sequential approach --> tf.keras.layers.InputLayer
# Functional approach --> tf.keras.Input

### Dense layer

In [ ]:
dense = tf.keras.layers.Dense(units=10, activation="softmax")

## Convolutional layer

In [ ]:
conv2d = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu")☻

## Pooling layers

In [ ]:
input = [[1.0,1.0],[1.0,1.0]]
input = tf.reshape(input, (1,2,2,1))

max_pool2d = tf.keras.layers.AvgPool2D()
output = max_pool2d(input)
print(output)

tf.Tensor([[[[1.]]]], shape=(1, 1, 1, 1), dtype=float32)


## Sequential class

In [ ]:
model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(28,28,1)),                    
            tf.keras.layers.Conv2D(16, (5,5)),
            tf.keras.layers.Dense(5)
])

model.add(tf.keras.layers.Dense(2))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 16)        416       
                                                                 
 dense (Dense)               (None, 24, 24, 5)         85        
                                                                 
 dense_1 (Dense)             (None, 24, 24, 2)         12        
                                                                 
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________


## Functional approach

In [ ]:
def get_model(nbr_filters=16, is_training=False):

  my_input = tf.keras.Input(shape=(28,28,1))
  my_input_2 = tf.keras.Input(shape=(100,100,1))

  x = tf.keras.layers.Conv2D(nbr_filters, (5,5))(my_input)
  x = tf.keras.layers.Dense(5)(x)
  x = tf.keras.layers.Dense(2)(x)

  model = tf.keras.Model(inputs=[my_input, my_input_2], outputs=x)

  return model

functional_model = get_model(32)

functional_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, 24, 24, 32)   832         ['input_2[0][0]']                
                                                                                                  
 dense_2 (Dense)                (None, 24, 24, 5)    165         ['conv2d_1[0][0]']               
                                                                                                  
 input_3 (InputLayer)           [(None, 100, 100, 1  0           []                               
                                )]                                                            

## Compiling a model

In [ ]:
functional_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=tf.keras.metrics.Accuracy())

## Training a model

In [ ]:
functional_model.fit()

## Making predictions with your model

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict
functional_model(x, is_training=False)
functional_model.predict(x)

## Model regularization

### Data augmentation using tensorflow layers

In [ ]:
def get_model(nbr_filters=16, is_training=False):

  my_input = tf.keras.Input(shape=(28,28,1))
  x = tf.keras.layers.Rescaling(scale=1./255)(my_input)

  if is_training:
    x = tf.keras.layers.RandomContrast(0.6)(x)

  x = tf.keras.layers.Conv2D(nbr_filters, (5,5))(my_input)
  x = tf.keras.layers.Dense(5)(x)
  x = tf.keras.layers.Dense(2)(x)

  model = tf.keras.Model(inputs=my_input, outputs=x)

  return model

## Data augmentation using tf.image 

In [ ]:
IMG_SIZE = 32

# create a function that does augmentation
def augment(image, label):

  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [IMG_SIZE,IMG_SIZE])
  image = (image/255.0)

  return image, label

# pass that function to your data inside .map method
my_dataset2 = (
    my_dataset2.map(augment)
    .batch(16)
)



## Data augmentation using generators

In [ ]:
from tf.keras.preprocessing.image import ImageDateGenerator

train_augmentor = ImageDateGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_augmentor.flow_from_directory(
    'data/images'
)

### Regularization using Dropout layer

In [ ]:
def get_model(nbr_filters=16, is_training=False):

  my_input = tf.keras.Input(shape=(28,28,1))

  x = tf.keras.layers.Conv2D(nbr_filters, (5,5))(my_input)

  if is_training:
    x = tf.keras.layers.Dropout(0.25)(x)

  x = tf.keras.layers.Dense(5)(x)
  x = tf.keras.layers.Dense(2)(x)

  model = tf.keras.Model(inputs=my_input, outputs=x)

  return model

## Batch normalization

In [ ]:
def get_model(nbr_filters=16, is_training=False):

  my_input = tf.keras.Input(shape=(28,28,1))

  x = tf.keras.layers.Conv2D(nbr_filters, (5,5))(my_input)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Dense(5)(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Dense(2)(x)

  model = tf.keras.Model(inputs=my_input, outputs=x)

  return model

## Callbacks

In [ ]:
from tf.keras.callbacks import ModelCheckpoint, LearningRateScheduler, Tensorboard

model_ckpt_callback = ModelCheckpoint(
    filepath="/data/models",
    save_weights_only=True,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True
)

my_model = get_model()

my_model.fit(
    # dataset,
    # other params,
    callbacks=[model_ckpt_callback]
)

my_model.evaluate()

my_model.predict()

## Saving and loading Tensorflow models

### Saving models using callbacks

In [ ]:
from tf.keras.callbacks import ModelCheckpoint

model_ckpt_callback = ModelCheckpoint(
    filepath="/data/models",
    save_weights_only=False,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True
)

my_model = tf.keras.models.load_model("/data/models")

my_model(x)



### Saving and loading model weights using save_weights method

In [ ]:
my_model = get_model()

my_model.fit(
    # dataset,
    # other params
    )

my_model.save_weights("/data/models_weights")

# Loading your model
my_model = get_model()
my_model.load_weights("/data/models_weights")




### Saving and loading entire Tensorflow models manually

In [ ]:
my_model = get_model()

my_model.fit(
    # dataset,
    # other params
    )

my_model.save("/data/entire_model")

# Load your model
my_model = tf.keras.models.load_model("/data/entire_model")
my_model(x)